Equality of Opportunity : TPR of group 1 = TPR of group 2

Equality violation = TPR group 1 - TPR Group 2

 ACSIncome_categories object (df_topandas expects a categories object) -- not used here though

In [1]:
from tabulate import tabulate


In [2]:
ACSIncome_categories = {
    "COW": {
        1.0: (
            "Employee of a private for-profit company or"
            "business, or of an individual, for wages,"
            "salary, or commissions"
        ),
        2.0: (
            "Employee of a private not-for-profit, tax-exempt,"
            "or charitable organization"
        ),
        3.0: "Local government employee (city, county, etc.)",
        4.0: "State government employee",
        5.0: "Federal government employee",
        6.0: (
            "Self-employed in own not incorporated business,"
            "professional practice, or farm"
        ),
        7.0: (
            "Self-employed in own incorporated business,"
            "professional practice or farm"
        ),
        8.0: "Working without pay in family business or farm",
        9.0: "Unemployed and last worked 5 years ago or earlier or never worked",
    },
    "SCHL": {
        1.0: "No schooling completed",
        2.0: "Nursery school, preschool",
        3.0: "Kindergarten",
        4.0: "Grade 1",
        5.0: "Grade 2",
        6.0: "Grade 3",
        7.0: "Grade 4",
        8.0: "Grade 5",
        9.0: "Grade 6",
        10.0: "Grade 7",
        11.0: "Grade 8",
        12.0: "Grade 9",
        13.0: "Grade 10",
        14.0: "Grade 11",
        15.0: "12th grade - no diploma",
        16.0: "Regular high school diploma",
        17.0: "GED or alternative credential",
        18.0: "Some college, but less than 1 year",
        19.0: "1 or more years of college credit, no degree",
        20.0: "Associate's degree",
        21.0: "Bachelor's degree",
        22.0: "Master's degree",
        23.0: "Professional degree beyond a bachelor's degree",
        24.0: "Doctorate degree",
    },
    "MAR": {
        1.0: "Married",
        2.0: "Widowed",
        3.0: "Divorced",
        4.0: "Separated",
        5.0: "Never married or under 15 years old",
    },
    "SEX": {1.0: "Male", 2.0: "Female"},
    "RAC1P": {
        1.0: "White alone",
        2.0: "Black or African American alone",
        3.0: "American Indian alone",
        4.0: "Alaska Native alone",
        5.0: (
            "American Indian and Alaska Native tribes specified;"
            "or American Indian or Alaska Native,"
            "not specified and no other"
        ),
        6.0: "Asian alone",
        7.0: "Native Hawaiian and Other Pacific Islander alone",
        8.0: "Some Other Race alone",
        9.0: "Two or More Races",
    },
}


This cell is custom task ACSIncome based on the Folktables paper, and is used in cell 
# [C]
group: RAC1P (Race)
target: POVPIP (Income Poverty Ratio Recode)

In [3]:
# """Data source and problem definitions for American Community Survey (ACS) Public Use Microdata Sample (PUMS)."""
import numpy as np
import pandas as pd

import folktables

ACSIncomePovertyRatio = folktables.BasicProblem(
    features=[
        'AGEP',
        'SCHL',
        'MAR',
        'SEX',
        'DIS',
        'ESP',
        'MIG',
        'CIT',
        'MIL',
        'ANC',
        'NATIVITY',
        'RELP',
        'DEAR',
        'DEYE',
        'DREM',
        'RAC1P',
        'GCL',
        'ESR',
        'OCCP',
        'WKHP',
    ],
    target='POVPIP',
    target_transform=lambda x: x < 250,
    group='RAC1P',
    preprocess=lambda x: x,
    postprocess=lambda x: np.nan_to_num(x, -1),
)



This cell is custom task ACSIncome based on the Folktables paper, and is used in cell 
# [A]

In [4]:
ACSIncome = folktables.BasicProblem(
    features=[
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'OCCP',
        'POBP',
        'RELP',
        'WKHP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    target_transform=lambda x: x > 50000,    
    group='RAC1P',
    preprocess=folktables.adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

This cell is custom task ACSIncomeNew, and is used in cell :
# [B]

It has all the same features as ACSIncome from the paper but it does not use AGEP (Age), the group is 'SEX' instead of 'RAC1P', and it predicts income > 25000, not 50000 like ACSIncome did.


In [5]:
import numpy as np
import pandas as pd

import folktables
ACSIncomeNew = folktables.BasicProblem(
    features=[
        'COW',
        'SCHL',
        'MAR',
        'OCCP',
        'POBP',
        'RELP',
        'WKHP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group='SEX',
    preprocess=folktables.adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

# [B]
**Group: 'SEX'for MF, prediction task ACSIncomeNew CA**
**CA -  ACSIncomeNew (income threshold  - $25,000 )**
9 features
target: PINCP
group: SEX for MF

individual's label is 1 if PINCP > 25000, 0 otherwise

In [6]:
from folktables import ACSDataSource, ACSEmployment, ACSIncome, generate_categories
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline


from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.impute import KNNImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split, KFold, ShuffleSplit

from scipy.stats import sem
from tqdm import tqdm


# import fairlearn
from fairlearn.metrics import equalized_odds_difference
from fairlearn.metrics import equalized_odds_ratio
from fairlearn.metrics import true_positive_rate, false_positive_rate

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# The ACS data source contains data for all fifty states, each of which has a
# slightly different distribution of features and response. This increases the
# diversity of environments in which we can evaluate our methods. For instance, we
# can generate another `ACSEmployment` task using data from Texas and repeat the
# experiment
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_ca = data_source.get_data(states=["CA"], download=True)

ca_features, ca_label, ca_group = ACSIncomeNew.df_to_numpy(acs_ca)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    ca_features, ca_label, ca_group, test_size=0.2, random_state=0)

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

yhat = model.predict(X_test)

#     library imports
ca_eq_odd_diff = equalized_odds_difference(y_test, yhat, sensitive_features=group_test)
ca_eq_odd_ratio = equalized_odds_ratio(y_test, yhat, sensitive_features=group_test)


male_tpr = np.mean(yhat[(y_test == 1) & (group_test == 1)])
female_tpr = np.mean(yhat[(y_test == 1) & (group_test == 2)])

male_tnr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 0)
female_tnr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 0)



male_fpr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 1)
female_fpr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 1)



male_fnr = np.mean(yhat[(y_test == 1) & (group_test == 1)] == 0)
female_fnr = np.mean(yhat[(y_test == 1) & (group_test == 2)] == 0)

male_accuracy = accuracy_score(y_test[group_test == 1], yhat[group_test == 1])
female_accuracy = accuracy_score(y_test[group_test == 2], yhat[group_test == 2])

# Equality of opportunity violation: 
ca_equality_violation = male_tpr - female_tpr



true_parity = np.abs(true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) - true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)]))
parity = (true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) + true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)])) / 2

printmd("## California Data")

printmd("**Predicting Income By Sex (M/F)**")

print("Number of samples for the 'male' group:", np.sum((group_test == 1)))


print("Number of samples for the 'female' group:", np.sum((group_test == 2)))
print("\n\n")

data = {
    'Metric': ['Equality Violation', 'Eq Odds Difference', 'Eq Odds Ratio',
               'True Parity', 'Parity',
               '(M-TPR)', '(F-TPR)',
               '(M-TNR)', '(F-TNR)',
               '(M-FPR)', '(F-FPR)',
               '(M-FNR)', '(F-FNR)',
               'Accuracy (Male)', 'Accuracy (Female)'],
    'Value': [ca_equality_violation, ca_eq_odd_diff, ca_eq_odd_ratio,
               true_parity, parity,
               male_tpr, female_tpr, male_tnr, female_tnr, male_fpr, female_fpr, male_fnr, female_fnr,
               male_accuracy, female_accuracy]
}
table_df = pd.DataFrame(data)

# Display the table
print(tabulate(table_df, showindex=False, headers=['Metric','Value'], tablefmt = "fancy_grid" ))


## California Data

**Predicting Income By Sex (M/F)**

Number of samples for the 'male' group: 20621
Number of samples for the 'female' group: 18512



╒════════════════════╤═══════════╕
│ Metric             │     Value │
╞════════════════════╪═══════════╡
│ Equality Violation │ 0.0469655 │
├────────────────────┼───────────┤
│ Eq Odds Difference │ 0.11779   │
├────────────────────┼───────────┤
│ Eq Odds Ratio      │ 0.752538  │
├────────────────────┼───────────┤
│ True Parity        │ 0.0469655 │
├────────────────────┼───────────┤
│ Parity             │ 0.892167  │
├────────────────────┼───────────┤
│ (M-TPR)            │ 0.91565   │
├────────────────────┼───────────┤
│ (F-TPR)            │ 0.868684  │
├────────────────────┼───────────┤
│ (M-TNR)            │ 0.524007  │
├────────────────────┼───────────┤
│ (F-TNR)            │ 0.641797  │
├────────────────────┼───────────┤
│ (M-FPR)            │ 0.475993  │
├────────────────────┼───────────┤
│ (F-FPR)            │ 0.358203  │
├────────────────────┼───────────┤
│ (M-FNR)            │ 0.084

# Analysis of Fairness Metrics:
==================================================

The above results show an **equality of opportunity violation** of approximately *0.047*.

This showing that the rate of TPR is slightly higher than the Black group. In other words, the Male group has a higher likelihood of getting a positive prediction given a ground truth positive label, compared with the Female group


The above results show an **equalized odds difference** of approximately *0.118*, showing the error rates are slightly higher for the Male group compared to the Female group


The above results show an **equalized odds ratio** of *0.75* which is interesting because this implies the error rates for positives and negatives are leaning more one way or another

The above results show an **true parity** of approximately *0.047*. It's slightly above 0 showing the error rates for classifying positive instances are slightly higher for the White group compared to the Female group. 

The above results show an **parity** of approximately *0.892*. It's closer to 1 than 0 showing the general error rates are significantly higher for the Male group compared to the Female group. 


===============================================================================

The above results show a *Male TPR* and *Female TPR* of approximately 91.5% and 86.9% respectively

The above results show a *Male TNR* and *Female TNR* of approximately 52.4% and 64.1% respectively

The above results show a *Male FPR* and *Female FPR* of approximately 47.5% and 35.8% respectively

The above results show a *Male FNR* and *Female FNR* of approximately 8.4% and 13.1% respectively

The above results show a *Male Accuracy* and *Female Accuracy* of approximately 80.5% and 75.2% respectively


The above results show a lot of similarities with the eduML findings. 

**TPR:*

For instance, like the eduML dataset, the results show that the Male TPR is higher than the Female TPR, with an almost 6% difference between the two. This indicates that there may be an inherent bias in the model's performance when classifiying positive instances in the Male ve Female group.

**TNR:**
The White TNR is actually significantly lower (~12%) than the Black TNR however, and there is higher disparity compared with the TPR, though both are large gaps. It does indicate some level of difference between model's performance with Male and Female groups in the dataset.


**FPR**
One interesting finding which echoes the eduML findings is the FPR gap. The Male FPR is slightly higher (1.3%) showing that White predictions for Income are more likely to get benefit of doubt for income level higher in the same way that White s

**FNR**

Another similarity with the eduML findings is the FNR gap. The Female FNR is significantly higher (5%) showing that Female groups higher than the income threshold are more likely to get a false prediction of being lower than the income threshold. 

This echoes the results from the eduML dataset and shows that some level of pre-processing still needs to be done to make the model more fair.





# [B]
**Group: 'SEX'for MF, prediction task ACSIncomeNew TX**
**TX -  ACSIncomeNew (income threshold  - $25,000 )**
9 features
target: PINCP
group: SEX for MF

individual's label is 1 if PINCP > 25000, 0 otherwise

In [7]:
from folktables import ACSDataSource, ACSEmployment, ACSIncome, generate_categories
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline


from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.impute import KNNImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split, KFold, ShuffleSplit

from scipy.stats import sem
from tqdm import tqdm


# import fairlearn
from fairlearn.metrics import equalized_odds_difference
from fairlearn.metrics import equalized_odds_ratio
from fairlearn.metrics import true_positive_rate, false_positive_rate

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# The ACS data source contains data for all fifty states, each of which has a
# slightly different distribution of features and response. This increases the
# diversity of environments in which we can evaluate our methods. For instance, we
# can generate another `ACSEmployment` task using data from Texas and repeat the
# experiment
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_tx = data_source.get_data(states=["TX"], download=True)

tx_features, tx_label, tx_group = ACSIncomeNew.df_to_numpy(acs_tx)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    tx_features, tx_label, tx_group, test_size=0.2, random_state=0)

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

yhat = model.predict(X_test)

#     library imports
tx_eq_odd_diff = equalized_odds_difference(y_test, yhat, sensitive_features=group_test)
tx_eq_odd_ratio = equalized_odds_ratio(y_test, yhat, sensitive_features=group_test)


male_tpr = np.mean(yhat[(y_test == 1) & (group_test == 1)])
female_tpr = np.mean(yhat[(y_test == 1) & (group_test == 2)])

male_tnr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 0)
female_tnr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 0)



male_fpr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 1)
female_fpr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 1)



male_fnr = np.mean(yhat[(y_test == 1) & (group_test == 1)] == 0)
female_fnr = np.mean(yhat[(y_test == 1) & (group_test == 2)] == 0)

male_accuracy = accuracy_score(y_test[group_test == 1], yhat[group_test == 1])
female_accuracy = accuracy_score(y_test[group_test == 2], yhat[group_test == 2])

# Equality of opportunity violation: 
tx_equality_violation = male_tpr - female_tpr



true_parity = np.abs(true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) - true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)]))
parity = (true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) + true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)])) / 2

printmd("## Texas Data")

printmd("**Predicting Income By Sex (M/F)**")

print("Number of samples for the 'male' group:", np.sum((group_test == 1)))


print("Number of samples for the 'female' group:", np.sum((group_test == 2)))
print("\n\n")

data = {
    'Metric': ['Equality Violation', 'Eq Odds Difference', 'Eq Odds Ratio',
               'True Parity', 'Parity',
               '(M-TPR)', '(F-TPR)',
               '(M-TNR)', '(F-TNR)',
               '(M-FPR)', '(F-FPR)',
               '(M-FNR)', '(F-FNR)',
               'Accuracy (Male)', 'Accuracy (Female)'],
    'Value': [tx_equality_violation, tx_eq_odd_diff, tx_eq_odd_ratio,
               true_parity, parity,
               male_tpr, female_tpr, male_tnr, female_tnr, male_fpr, female_fpr, male_fnr, female_fnr,
               male_accuracy, female_accuracy]
}
table_df = pd.DataFrame(data)

# Display the table
print(tabulate(table_df, showindex=False, headers=['Metric','Value'], tablefmt = "fancy_grid" ))


## Texas Data

**Predicting Income By Sex (M/F)**

Number of samples for the 'male' group: 14495
Number of samples for the 'female' group: 12690



╒════════════════════╤═══════════╕
│ Metric             │     Value │
╞════════════════════╪═══════════╡
│ Equality Violation │ 0.0736927 │
├────────────────────┼───────────┤
│ Eq Odds Difference │ 0.125046  │
├────────────────────┼───────────┤
│ Eq Odds Ratio      │ 0.734393  │
├────────────────────┼───────────┤
│ True Parity        │ 0.0736927 │
├────────────────────┼───────────┤
│ Parity             │ 0.879038  │
├────────────────────┼───────────┤
│ (M-TPR)            │ 0.915884  │
├────────────────────┼───────────┤
│ (F-TPR)            │ 0.842192  │
├────────────────────┼───────────┤
│ (M-TNR)            │ 0.529209  │
├────────────────────┼───────────┤
│ (F-TNR)            │ 0.654254  │
├────────────────────┼───────────┤
│ (M-FPR)            │ 0.470791  │
├────────────────────┼───────────┤
│ (F-FPR)            │ 0.345746  │
├────────────────────┼───────────┤
│ (M-FNR)            │ 0.084

# Analysis of Fairness Metrics:
==================================================

The above results show an **equality of opportunity violation** of approximately *0.073*.

This showing that the rate of TPR is slightly higher than the Black group. In other words, the Male group has a higher likelihood of getting a positive prediction given a ground truth positive label, compared with the Female group


The above results show an **equalized odds difference** of approximately *0.125*, showing the error rates are higher for the Male group compared to the Female group


The above results show an **equalized odds ratio** of *0.73* which extremely high implying that there is an increased error rate either on the positive or negative side

The above results show an **true parity** of approximately *0.074*. It's slightly above 0 showing the error rates for classifying positive instances are slightly higher for the Male group compared to the Female group. 

The above results show an **parity** of approximately *0.88*. It's closer to 1 than 0 showing the general error rates are significantly higher for the Male group compared to the Female group. 


===============================================================================

The above results show a *Male TPR* and *Female TPR* of approximately 91.5% and 84.2% respectively

The above results show a *Male TNR* and *Female TNR* of approximately 47.1% and 34.6% respectively

The above results show a *Male FPR* and *Female FPR* of approximately 47.1% and 34.6% respectively

The above results show a *Male FNR* and *Female FNR* of approximately 25.0% and 34.5% respectively

The above results show a *Male Accuracy* and *Female Accuracy* of approximately 78.6% and 75.2% respectively


The above results show a lot of similarities with the eduML findings. 

**TPR:*

For instance, like the eduML dataset, the results show that the White TPR is higher than the Black TPR, with an almost 10% difference between the two. This indicates that there may be an inherent bias in the model's performance when classifiying positive instances in the White ve Black group.

**TNR:**
The White TNR is actually lower (~1%) than the Black TNR however, but there is lesser disparity compared with the TPR. It does indicate some level of difference between model's performance with Black and White groups in the dataset.


**FPR**
One interesting finding which echoes the eduML findings is the FPR gap. The White FPR is slightly higher (1.3%) showing that White predictions for Income are more likely to get benefit of doubt for income level higher in the same way that White s

**FNR**

Another similarity with the eduML findings is the FNR gap. The Black FNR is significantly higher (11.4%) showing that Black groups higher than the income threshold are more likely to get a false prediction of being lower than the income threshold. 

This echoes the results from the eduML dataset and shows that some level of pre-processing still needs to be done to make the model more fair.






# [A]

**CA -  ACSIncome (income threshold  - $50,000 )**
10 features
target: PINCP
group: RAC1P for BW

individual's label is 1 if PINCP > 50000, 0 otherwise

(imports and data download are in previous cell)

In [8]:
# ca label, ca_features, ca_group, be able to explain what that value is and what values from 1-8 for example in CA_group
# do histogram to understand each specific understanding of what they are
# really good understanding of data for data analysis
# which features are useful for group label, predictions
# nan value because denominator is 0 and group label might not be 1 and 2
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

    
ca_features, ca_label, ca_group = ACSIncome.df_to_numpy(acs_ca)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    ca_features, ca_label, ca_group, test_size=0.2, random_state=0)

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

yhat = model.predict(X_test)


    
#     library imports
ca_eq_odd_diff = equalized_odds_difference(y_test, yhat, sensitive_features=group_test)
ca_eq_odd_ratio = equalized_odds_ratio(y_test, yhat, sensitive_features=group_test)


white_tpr = np.mean(yhat[(y_test == 1) & (group_test == 1)])
black_tpr = np.mean(yhat[(y_test == 1) & (group_test == 2)])

white_tnr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 0)
black_tnr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 0)



white_fpr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 1)
black_fpr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 1)



white_fnr = np.mean(yhat[(y_test == 1) & (group_test == 1)] == 0)
black_fnr = np.mean(yhat[(y_test == 1) & (group_test == 2)] == 0)

white_accuracy = accuracy_score(y_test[group_test == 1], yhat[group_test == 1])
black_accuracy = accuracy_score(y_test[group_test == 2], yhat[group_test == 2])

# Equality of opportunity violation: 
ca_equality_violation = white_tpr - black_tpr



true_parity = np.abs(true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) - true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)]))
parity = (true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) + true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)])) / 2

printmd("## California Data")

printmd("**Predicting Income By Race (B/W)**")

print("Number of samples for the 'white' group:", np.sum((group_test == 1)))


print("Number of samples for the 'black' group:", np.sum((group_test == 2)))
print("\n\n")

data = {
    'Metric': ['Equality Violation', 'Eq Odds Difference', 'Eq Odds Ratio',
               'True Parity', 'Parity',
               '(W-TPR)', '(B-TPR)',
               '(W-TNR)', '(B-TNR)',
               '(W-FPR)', '(B-FPR)',
               '(W-FNR)', '(B-FNR)',
               'Accuracy (White)', 'Accuracy (Black)'],
    'Value': [ca_equality_violation, ca_eq_odd_diff, ca_eq_odd_ratio,
               true_parity, parity,
               white_tpr, black_tpr, white_tnr, black_tnr, white_fpr, black_fpr, white_fnr, black_fnr,
               white_accuracy, black_accuracy]
}
table_df = pd.DataFrame(data)

# Display the table
print(tabulate(table_df, showindex=False, headers=['Metric','Value'], tablefmt = "fancy_grid" ))


## California Data

**Predicting Income By Race (B/W)**

Number of samples for the 'white' group: 24197
Number of samples for the 'black' group: 1698



╒════════════════════╤═══════════╕
│ Metric             │     Value │
╞════════════════════╪═══════════╡
│ Equality Violation │ 0.0944656 │
├────────────────────┼───────────┤
│ Eq Odds Difference │ 0.362903  │
├────────────────────┼───────────┤
│ Eq Odds Ratio      │ 0         │
├────────────────────┼───────────┤
│ True Parity        │ 0.0944656 │
├────────────────────┼───────────┤
│ Parity             │ 0.702627  │
├────────────────────┼───────────┤
│ (W-TPR)            │ 0.74986   │
├────────────────────┼───────────┤
│ (B-TPR)            │ 0.655395  │
├────────────────────┼───────────┤
│ (W-TNR)            │ 0.814397  │
├────────────────────┼───────────┤
│ (B-TNR)            │ 0.807799  │
├────────────────────┼───────────┤
│ (W-FPR)            │ 0.185603  │
├────────────────────┼───────────┤
│ (B-FPR)            │ 0.192201  │
├────────────────────┼───────────┤
│ (W-FNR)            │ 0.2501

# Analysis of Fairness Metrics:
==================================================

The above results show an **equality of opportunity violation** of approximately *0.094*.

This showing that the rate of TPR is slightly higher than the Black group. In other words, the White group has a higher likelihood of getting a positive prediction given a ground truth positive label, compared with the Black group


The above results show an **equalized odds difference** of approximately *0.362*, showing the error rates are slightly higher for the White group compared to the Black group


The above results show an **equalized odds ratio** of *0* which is interesting because this implies the error rates for positives and negatives are pretty much the same.

The above results show an **true parity** of approximately *0.094*. It's slightly above 0 showing the error rates for classifying positive instances are slightly higher for the White group compared to the Black group. 

The above results show an **parity** of approximately *0.702*. It's closer to 1 than 0 showing the general error rates are significantly higher for the White group compared to the Black group. 


===============================================================================

The above results show a *White TPR* and *Black TPR* of approximately 74.9% and 65.5% respectively

The above results show a *White TNR* and *Black TNR* of approximately 18.6% and 19.2% respectively

The above results show a *White FPR* and *Black FPR* of approximately 18.6% and 19.2% respectively

The above results show a *White FNR* and *Black FNR* of approximately 25.0% and 34.5% respectively

The above results show a *White Accuracy* and *Black Accuracy* of approximately 78.6% and 75.2% respectively


The above results show a lot of similarities with the eduML findings. 

**TPR:*

For instance, like the eduML dataset, the results show that the White TPR is higher than the Black TPR, with an almost 10% difference between the two. This indicates that there may be an inherent bias in the model's performance when classifiying positive instances in the White ve Black group.

**TNR:**
The White TNR is actually lower (~1%) than the Black TNR however, but there is lesser disparity compared with the TPR. It does indicate some level of difference between model's performance with Black and White groups in the dataset.


**FPR**
One interesting finding which echoes the eduML findings is the FPR gap. The White FPR is slightly higher (1.3%) showing that White predictions for Income are more likely to get benefit of doubt for income level higher in the same way that White s

**FNR**

Another similarity with the eduML findings is the FNR gap. The Black FNR is significantly higher (11.4%) showing that Black groups higher than the income threshold are more likely to get a false prediction of being lower than the income threshold. 

This echoes the results from the eduML dataset and shows that some level of pre-processing still needs to be done to make the model more fair.





# [A]

**TX -  ACSIncome (income threshold  - $50,000 )**
10 features
target: PINCP
Group: RAC1P for BW
individual's label is 1 if PINCP > 50000, 0 otherwise

In [9]:
from folktables import ACSDataSource, ACSEmployment, ACSIncome, generate_categories
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from tabulate import tabulate


from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.impute import KNNImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split, KFold, ShuffleSplit

from scipy.stats import sem
from tqdm import tqdm


# import fairlearn
from fairlearn.metrics import equalized_odds_difference
from fairlearn.metrics import equalized_odds_ratio
from fairlearn.metrics import true_positive_rate, false_positive_rate


from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# The ACS data source contains data for all fifty states, each of which has a
# slightly different distribution of features and response. This increases the
# diversity of environments in which we can evaluate our methods. For instance, we
# can generate another `ACSEmployment` task using data from Texas and repeat the
# experiment
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')


acs_tx = data_source.get_data(states=["TX"], download=True)
tx_features, tx_label, tx_group = ACSIncome.df_to_numpy(acs_tx)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    tx_features, tx_label, tx_group, test_size=0.2, random_state=0)

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

yhat = model.predict(X_test)


                            
# library imports
tx_eq_odd_diff = equalized_odds_difference(y_test, yhat, sensitive_features=group_test)
tx_eq_odd_ratio = equalized_odds_ratio(y_test, yhat, sensitive_features=group_test)

white_tpr = np.mean(yhat[(y_test == 1) & (group_test == 1)])
black_tpr = np.mean(yhat[(y_test == 1) & (group_test == 2)])

white_tnr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 0)
black_tnr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 0)

white_fpr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 1)
black_fpr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 1)

white_fnr = np.mean(yhat[(y_test == 1) & (group_test == 1)] == 0)
black_fnr = np.mean(yhat[(y_test == 1) & (group_test == 2)] == 0)

white_accuracy = accuracy_score(y_test[group_test == 1], yhat[group_test == 1])
black_accuracy = accuracy_score(y_test[group_test == 2], yhat[group_test == 2])


# Equality of opportunity violation:
tx_equality_violation = white_tpr - black_tpr



true_parity = np.abs(true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) - true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)]))
parity = (true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) + true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)])) / 2


printmd("## Texas Data")

printmd("**Predicting Income By Race (B/W)**")

print("Number of samples for the 'white' group:", np.sum((group_test == 1)))


print("Number of samples for the 'black' group:", np.sum((group_test == 2)))
print("\n\n")

data = {
    'Metric': ['Equality Violation', 'Eq Odds Difference', 'Eq Odds Ratio',
               'True Parity', 'Parity',
               '(W-TPR)', '(B-TPR)',
               '(W-TNR)', '(B-TNR)',
               '(W-FPR)', '(B-FPR)',
               '(W-FNR)', '(B-FNR)',
               'Accuracy (White)', 'Accuracy (Black)'],
    'Value': [tx_equality_violation, tx_eq_odd_diff, tx_eq_odd_ratio,
               true_parity, parity,
               white_tpr, black_tpr, white_tnr, black_tnr, white_fpr, black_fpr, white_fnr, black_fnr,
               white_accuracy, black_accuracy]
}
table_df = pd.DataFrame(data)

# Display the table
print(tabulate(table_df, showindex=False, headers=['Metric','Value'], tablefmt = "fancy_grid" ))



## Texas Data

**Predicting Income By Race (B/W)**

Number of samples for the 'white' group: 21184
Number of samples for the 'black' group: 2467



╒════════════════════╤══════════╕
│ Metric             │    Value │
╞════════════════════╪══════════╡
│ Equality Violation │ 0.132597 │
├────────────────────┼──────────┤
│ Eq Odds Difference │ 0.39469  │
├────────────────────┼──────────┤
│ Eq Odds Ratio      │ 0.241306 │
├────────────────────┼──────────┤
│ True Parity        │ 0.132597 │
├────────────────────┼──────────┤
│ Parity             │ 0.598056 │
├────────────────────┼──────────┤
│ (W-TPR)            │ 0.664355 │
├────────────────────┼──────────┤
│ (B-TPR)            │ 0.531758 │
├────────────────────┼──────────┤
│ (W-TNR)            │ 0.851126 │
├────────────────────┼──────────┤
│ (B-TNR)            │ 0.869832 │
├────────────────────┼──────────┤
│ (W-FPR)            │ 0.148874 │
├────────────────────┼──────────┤
│ (B-FPR)            │ 0.130168 │
├────────────────────┼──────────┤
│ (W-FNR)            │ 0.335645 │
├───────────────────

# Analysis of Fairness Metrics:
==================================================

The above results show an **equality of opportunity violation** of approximately *0.132*.

This showing that the rate of TPR / accurately predicted positives juxtaposed with ground truth positives from the training data for the White group is significantly higher than the Black group. In other words, the White group has a higher likelihood of getting a positive prediction given a ground truth positive label, compared with the Black group


The above results show an **equalized odds difference** of approximately *0.394*. It's slightly above 0 showing the error rates are slightly higher for the White group compared to the Black group


The above results show an **equalized odds ratio** of approximately *0.241*. It's slightly above 0 showing the error rates are slightly higher for the White group compared to the Black group. 

The above results show an **true parity** of approximately *0.132*. It's slightly above 0 showing the error rates for classifying positive instances are slightly higher for the White group compared to the Black group. 

The above results show an **parity** of approximately *0.598*. It's well above 0 and closer to 1 showing the general error rates are significantly higher for the White group compared to the Black group. 


===============================================================================

The above results show a *White TPR* and *Black TPR* of approximately 66.5% and 53.1% respectively

The above results show a *White TNR* and *Black TNR* of approximately 85.1% and 86.9% respectively

The above results show a *White FPR* and *Black FPR* of approximately 14.9% and 13.0% respectively

The above results show a *White FNR* and *Black FNR* of approximately 33.6% and 46.8% respectively

The above results show a *White Accuracy* and *Black Accuracy* of approximately 77.8% and 77.7% respectively


The above results show a lot of similarities with the eduML findings. 

**TPR:*

For instance, like the eduML dataset, the results show that the White TPR is higher than the Black TPR, with an almost 13% difference between the two. This indicates that there may be an inherent bias in the model's performance when classifiying positive instances in the White ve Black group.

**TNR:**
The White TNR is actually lower (~1.8%) than the Black TNR however, but there is lesser disparity compared with the TPR. It does indicate some level of difference between model's performance with Black and White groups in the dataset and that prediction of negatives is lower for the White group (possible bias).


**FPR**
One interesting finding which echoes the eduML findings is the FPR gap. The White FPR is slightly higher (1.8%) showing that White predictions for Income are more likely to get benefit of doubt for income level higher in the same way that White students were more likely to get benefit of the doubt for positive predictions for readiness for advanced math in the MSS/HSLS datasets.

**FNR**

Another similarity with the eduML findings is the FNR gap. The Black FNR is significantly higher (13.3%) showing that Black groups higher than the income threshold are more likely to get a false prediction of being lower than the income threshold. This parallels the eduML dataset and findings in that Black students were also more likely to be falsely predicted to do poorly in advanced math even when ground truth showed they would do well.

This echoes the results from the eduML dataset and shows that some level of pre-processing still needs to be done to make the model more fair.





# [C]

**TX -  ACSIncomePovertyRatio (income threshold  - $250 )**
10 features
target: POVPIP
Group: RAC1P for BW
individual's label is 1 if PINCP > 250, 0 otherwise
**Group: 'RAC1P'for BW, prediction task ACSIncomePovertyRatio TX**
group: RAC1P (Race)
target: POVPIP (Income Poverty Ratio Recode)

In [10]:
from folktables import ACSDataSource, ACSEmployment, ACSIncome, generate_categories
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.impute import KNNImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split, KFold, ShuffleSplit
from sklearn import metrics

from scipy.stats import sem
from tqdm import tqdm


# import fairlearn
from fairlearn.metrics import equalized_odds_difference
from fairlearn.metrics import equalized_odds_ratio
from fairlearn.metrics import true_positive_rate, false_positive_rate

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

    

# The ACS data source contains data for all fifty states, each of which has a
# slightly different distribution of features and response. This increases the
# diversity of environments in which we can evaluate our methods. For instance, we
# can generate another `ACSEmployment` task using data from Texas and repeat the
# experiment
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')


acs_tx = data_source.get_data(states=["TX"], download=True)
tx_features, tx_label, tx_group = ACSIncomePovertyRatio.df_to_numpy(acs_tx)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    tx_features, tx_label, tx_group, test_size=0.2, random_state=0)

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

yhat = model.predict(X_test)


    
    #library imports
tx_eq_odd_diff = equalized_odds_difference(y_test, yhat, sensitive_features=group_test)
tx_eq_odd_ratio = equalized_odds_ratio(y_test, yhat, sensitive_features=group_test)


white_tpr = np.mean(yhat[(y_test == 1) & (group_test == 1)])
black_tpr = np.mean(yhat[(y_test == 1) & (group_test == 2)])

white_tnr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 0)
black_tnr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 0)



white_fpr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 1)
black_fpr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 1)



white_fnr = np.mean(yhat[(y_test == 1) & (group_test == 1)] == 0)
black_fnr = np.mean(yhat[(y_test == 1) & (group_test == 2)] == 0)

white_accuracy = accuracy_score(y_test[group_test == 1], yhat[group_test == 1])
black_accuracy = accuracy_score(y_test[group_test == 2], yhat[group_test == 2])

# Equality of opportunity violation: 
tx_equality_violation = np.abs(white_tpr - black_tpr)



true_parity = np.abs(true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) - true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)]))
parity = (true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) + true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)])) / 2

printmd("## Texas Data")

printmd("**Predicting Income Poverty Ratio By Race (B/W)**")

print("Number of samples for the 'white' group:", np.sum((group_test == 1)))

# print("Number of samples for the 'black' group:", np.sum((y_test == 1) & (group_test == 2)))

print("Number of samples for the 'black' group:", np.sum((group_test == 2)))
print("\n\n")

data = {
    'Metric': ['Equality Violation', 'Eq Odds Difference', 'Eq Odds Ratio',
               'True Parity', 'Parity',
               '(W-TPR)', '(B-TPR)',
               '(W-TNR)', '(B-TNR)',
               '(W-FPR)', '(B-FPR)',
               '(W-FNR)', '(B-FNR)',
               'Accuracy (White)', 'Accuracy (Black)'],
    'Value': [tx_equality_violation, tx_eq_odd_diff, tx_eq_odd_ratio,
               true_parity, parity,
               white_tpr, black_tpr, white_tnr, black_tnr, white_fpr, black_fpr, white_fnr, black_fnr,
               white_accuracy, black_accuracy]
}
table_df = pd.DataFrame(data)

# Display the table
print(tabulate(table_df, showindex=False, headers=['Metric','Value'], tablefmt = "fancy_grid" ))

## Texas Data

**Predicting Income Poverty Ratio By Race (B/W)**

Number of samples for the 'white' group: 41739
Number of samples for the 'black' group: 5072



╒════════════════════╤═══════════╕
│ Metric             │     Value │
╞════════════════════╪═══════════╡
│ Equality Violation │ 0.0731338 │
├────────────────────┼───────────┤
│ Eq Odds Difference │ 0.529032  │
├────────────────────┼───────────┤
│ Eq Odds Ratio      │ 0         │
├────────────────────┼───────────┤
│ True Parity        │ 0.0731338 │
├────────────────────┼───────────┤
│ Parity             │ 0.429779  │
├────────────────────┼───────────┤
│ (W-TPR)            │ 0.393212  │
├────────────────────┼───────────┤
│ (B-TPR)            │ 0.466346  │
├────────────────────┼───────────┤
│ (W-TNR)            │ 0.872892  │
├────────────────────┼───────────┤
│ (B-TNR)            │ 0.832974  │
├────────────────────┼───────────┤
│ (W-FPR)            │ 0.127108  │
├────────────────────┼───────────┤
│ (B-FPR)            │ 0.167026  │
├────────────────────┼───────────┤
│ (W-FNR)            │ 0.6067

# Analysis of Fairness Metrics:
==================================================

The above results show an **equality of opportunity violation** of approximately *0.073*.

This shows the TPR difference and that the White group has a slightly higher likelihood of getting a positive prediction given a ground truth positive label, compared with the Black group


The above results show an **equalized odds difference** of approximately *0.529*. It's halfway between 0 and 1 showing the error rates for positives and negatives are equalized for the White group and the Black group


The above results show an **equalized odds ratio** is exactly *0* which is interesting because this implies error rates are equalized in the positive and negative direction

The above results show an **true parity** of approximately *0.073*. It's slightly above 0 showing the error rates for classifying positive instances are slightly higher for the White group compared to the Black group. 

The above results show an **parity** of approximately *0.429*. It's well above 0 and closer to 0.5 showing the general error rates are somewhat equalized for positives and negatives for the White group and for the Black group. 


===============================================================================

The above results show a *White TPR* and *Black TPR* of approximately 39.3% and 46.6% respectively

The above results show a *White TNR* and *Black TNR* of approximately 87.3% and 83.3% respectively

The above results show a *White FPR* and *Black FPR* of approximately 12.7% and 16.7% respectively

The above results show a *White FNR* and *Black FNR* of approximately 60.7% and 53.4% respectively

The above results show a *White Accuracy* and *Black Accuracy* of approximately 70.2% and 66.7% respectively


The above results show a lot of similarities with the eduML findings. 

**TPR:*

For instance, like the eduML dataset, the results show that the White TPR is higher than the Black TPR, with an almost 7% difference between the two. This indicates that there may be an inherent bias in the model's performance when classifiying positive instances in the White ve Black group.

**TNR:**
The White TNR is moderately higher (4%) than the Black TNR however, and there is comparable gaps disparity with the TPR. It does indicate some level of difference between model's performance with Black and White groups in the dataset.


**FPR**
One interesting finding which differs from the eduML findings is the FPR gap. The White FPR is actually a lot lower (4%) showing that White predictions for Income are actually less likely to get benefit of doubt for income level higher, whereas in the eduML dataset the opposite was true.

**FNR**

Another similarity with the eduML findings is the FNR gap. The Black FNR is actually higher (7.3%) showing that White groups higher than the income threshold are more likely to get a false prediction of being lower than the income threshold. 

This shows results which are opposite from the eduML dataset which is quite interesting as White group is penalized more but the White group seems to have a higher accuracy than the Black group.




# [C]

**CA -  ACSIncomePovertyRatio (income threshold  - $250 )**
10 features
target: POVPIP
Group: RAC1P for BW
individual's label is 1 if PINCP > 250, 0 otherwise
**Group: 'RAC1P'for BW, prediction task ACSIncomePovertyRatio CA**


In [11]:
from folktables import ACSDataSource, ACSEmployment, ACSIncome, generate_categories
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.impute import KNNImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split, KFold, ShuffleSplit

from scipy.stats import sem
from tqdm import tqdm


# import fairlearn
from fairlearn.metrics import equalized_odds_difference
from fairlearn.metrics import equalized_odds_ratio
from fairlearn.metrics import true_positive_rate, false_positive_rate

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))


# The ACS data source contains data for all fifty states, each of which has a
# slightly different distribution of features and response. This increases the
# diversity of environments in which we can evaluate our methods. For instance, we
# can generate another `ACSEmployment` task using data from Texas and repeat the
# experiment
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')


acs_ca = data_source.get_data(states=["CA"], download=True)
ca_features, ca_label, ca_group = ACSIncomePovertyRatio.df_to_numpy(acs_ca)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    ca_features, ca_label, ca_group, test_size=0.2, random_state=0)

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

yhat = model.predict(X_test)


    
#     library imports
ca_eq_odd_diff = equalized_odds_difference(y_test, yhat, sensitive_features=group_test)
ca_eq_odd_ratio = equalized_odds_ratio(y_test, yhat, sensitive_features=group_test)


white_tpr = np.mean(yhat[(y_test == 1) & (group_test == 1)])
black_tpr = np.mean(yhat[(y_test == 1) & (group_test == 2)])

white_tnr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 0)
black_tnr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 0)



white_fpr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 1)
black_fpr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 1)



white_fnr = np.mean(yhat[(y_test == 1) & (group_test == 1)] == 0)
black_fnr = np.mean(yhat[(y_test == 1) & (group_test == 2)] == 0)

white_accuracy = accuracy_score(y_test[group_test == 1], yhat[group_test == 1])
black_accuracy = accuracy_score(y_test[group_test == 2], yhat[group_test == 2])

# Equality of opportunity violation: 
ca_equality_violation = np.abs(white_tpr - black_tpr)



true_parity = np.abs(true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) - true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)]))
parity = (true_positive_rate(y_test[(group_test == 1)], yhat[(group_test == 1)]) + true_positive_rate(y_test[(group_test == 2)], yhat[(group_test == 2)])) / 2

printmd("## California Data")

printmd("**Predicting Income Poverty Ratio By Race (B/W)**")

print("Number of samples for the 'white' group:", np.sum((group_test == 1)))

# print("Number of samples for the 'black' group:", np.sum((y_test == 1) & (group_test == 2)))

print("Number of samples for the 'black' group:", np.sum((group_test == 2)))
print("\n\n")

data = {
    'Metric': ['Equality Violation', 'Eq Odds Difference', 'Eq Odds Ratio',
               'True Parity', 'Parity',
               '(W-TPR)', '(B-TPR)',
               '(W-TNR)', '(B-TNR)',
               '(W-FPR)', '(B-FPR)',
               '(W-FNR)', '(B-FNR)',
               'Accuracy (White)', 'Accuracy (Black)'],
    'Value': [ca_equality_violation, ca_eq_odd_diff, ca_eq_odd_ratio,
               true_parity, parity,
               white_tpr, black_tpr, white_tnr, black_tnr, white_fpr, black_fpr, white_fnr, black_fnr,
               white_accuracy, black_accuracy]
}
table_df = pd.DataFrame(data)

# Display the table
print(tabulate(table_df, showindex=False, headers=['Metric','Value'], tablefmt = "fancy_grid" ))

                              



  

## California Data

**Predicting Income Poverty Ratio By Race (B/W)**

Number of samples for the 'white' group: 46245
Number of samples for the 'black' group: 3691



╒════════════════════╤═══════════╕
│ Metric             │     Value │
╞════════════════════╪═══════════╡
│ Equality Violation │ 0.0494294 │
├────────────────────┼───────────┤
│ Eq Odds Difference │ 0.456793  │
├────────────────────┼───────────┤
│ Eq Odds Ratio      │ 0         │
├────────────────────┼───────────┤
│ True Parity        │ 0.0494294 │
├────────────────────┼───────────┤
│ Parity             │ 0.326405  │
├────────────────────┼───────────┤
│ (W-TPR)            │ 0.301691  │
├────────────────────┼───────────┤
│ (B-TPR)            │ 0.35112   │
├────────────────────┼───────────┤
│ (W-TNR)            │ 0.911361  │
├────────────────────┼───────────┤
│ (B-TNR)            │ 0.88081   │
├────────────────────┼───────────┤
│ (W-FPR)            │ 0.0886388 │
├────────────────────┼───────────┤
│ (B-FPR)            │ 0.11919   │
├────────────────────┼───────────┤
│ (W-FNR)            │ 0.6983

# Analysis of Fairness Metrics:
==================================================

The above results show an **equality of opportunity violation** of approximately *0.049*.

This shows the TPR difference and that the White group has a slightly higher likelihood of getting a positive prediction given a ground truth positive label, compared with the Black group


The above results show an **equalized odds difference** of approximately *0.457*. It's halfway between 0 and 1 showing the error rates for positives and negatives are equalized for the White group and the Black group


The above results show an **equalized odds ratio** is exactly *0* which is interesting because this implies error rates are equalized in the positive and negative direction

The above results show an **true parity** of approximately *0.049*. It's slightly above 0 showing the error rates for classifying positive instances are slightly higher for the White group compared to the Black group. 

The above results show an **parity** of approximately *0.326*. It's above 0 and closer to 0.4 showing the general error rates are somewhat equalized for positives and negatives for the White group and for the Black group. 


===============================================================================

The above results show a *White TPR* and *Black TPR* of approximately 30.1% and 35.1% respectively

The above results show a *White TNR* and *Black TNR* of approximately 91.1% and 88.1% respectively

The above results show a *White FPR* and *Black FPR* of approximately 8.86% and 11.9% respectively

The above results show a *White FNR* and *Black FNR* of approximately 69.8% and 64.9% respectively

The above results show a *White Accuracy* and *Black Accuracy* of approximately 71.8% and 66.3% respectively


The above results show a lot of similarities with the eduML findings. 

**TPR:**

For instance, like the eduML dataset, the results show that the White TPR is actually lower than the Black TPR, with an almost 5% difference between the two. This indicates that there may be an inherent bias in the model's performance when classifying positive instances in the White ve Black group.

**TNR:**
The White TNR is moderately higher (3.1%) than the Black TNR however, and there is comparable gaps disparity with the TPR. It does indicate some level of difference between model's performance with Black and White groups in the dataset.


**FPR**
One interesting finding which differs from the eduML findings is the FPR gap. The White FPR is actually a lot lower (3.1%) showing that White predictions for Income are actually less likely to get benefit of doubt for income level higher, whereas in the eduML dataset the opposite was true.

**FNR**

Another similarity with the eduML findings is the FNR gap. The Black FNR is  lower (3%) showing that White groups higher than the income threshold are more likely to get a false prediction of being lower than the income threshold. 

This shows results which are opposite from the eduML dataset which is quite interesting as White group is penalized more but the White group seems to have a higher accuracy than the Black group.


